In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import scipy as scp

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_data = pd.read_csv("/kaggle/input/poker-rule-induction/train.csv.zip")
train_data.head()




In [ ]:
test_data = pd.read_csv("/kaggle/input/poker-rule-induction/test.csv.zip")
test_data.head()

In [ ]:
df = train_data
df.head()

#S1 “Suit of card #1”
#C1 “Rank of card #1”
#Ordinal (1-4) representing {Hearts, Spades, Diamonds, Clubs}
#Numerical (1-13) representing (Ace, 2, 3, .. , Queen, King)""" 

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.hist(figsize = (16,12));

In [ ]:
df['hand'].value_counts()

# Func

In [ ]:
def preproc(df):
    df['min-flush'] = (df['S1']==df['S2'])&(df['S2']==df['S3'])&(df['S3']==df['S4'])&(df['S4']==df['S5'])
    a1 = df[['C1', 'C2', 'C3', 'C4', 'C5']].min(axis = 1)
    b1 = 5*a1 + 10
    a2 = df[['C1', 'C2', 'C3', 'C4', 'C5']].max(axis = 1).apply(scp.math.factorial)
    b2 = (df[['C1', 'C2', 'C3', 'C4', 'C5']].min(axis = 1) -1 ).apply(scp.math.factorial)
    c = a2/b2
    df['min-straight'] = ((df['C1']*df['C2']*df['C3']*df['C4']*df['C5'] == c) & (df['C1']+df['C2']+df['C3']+df['C4']+df['C5'] == b1)) | ( (df[['C1', 'C2', 'C3', 'C4', 'C5']].min(axis = 1) == 1) & (df['C1']*df['C2']*df['C3']*df['C4']*df['C5'] == 17160))
    df['min - straight flush'] = df['min-flush'] & df['min-straight']
    df['RF'] = df[df['min - straight flush']==True][['C1', 'C2', 'C3','C4', 'C5']].sum(axis =1) == 47
    df['RF'] = df['RF'].fillna(False)
    df['straight flush'] = (df['min - straight flush'] ==True) & (df['RF'] ==False)
    df['straight flush'] = df['straight flush'].fillna(False)
    df = df.drop('min - straight flush', axis = 1)
    df['flush'] = (df['straight flush'] == False) & (df['min-flush'] ==True) & (df['RF'] ==False)
    df['straight'] = (df['straight flush'] == False) & (df['min-straight'] ==True) & (df['RF'] ==False)
    df = df.drop('min-straight', axis = 1)
    df = df.drop('min-flush', axis = 1)
    df['flush'] = df['flush'].fillna(False)
    df['straight'] = df['straight'].fillna(False)
    df['Nothing in hand'] = (df[['C1', 'C2', 'C3', 'C4', 'C5']].nunique(axis =1) == 5) & (df['straight'] ==False) & (df['flush'] == False) & (df['RF'] == False)& (df['straight flush'] == False) 
    df['Nothing in hand'] = df['Nothing in hand'].fillna(False)
    df['One pair'] = (df[['C1', 'C2', 'C3', 'C4', 'C5']].nunique(axis =1) == 4)
    df['One pair'] = df['One pair'].fillna(False)
    df['Four of a kind or Full house'] = df[['C1', 'C2', 'C3', 'C4', 'C5']].nunique(axis =1) == 2
    df['Two pairs or Three of a kind'] = df[['C1', 'C2', 'C3', 'C4', 'C5']].nunique(axis =1) == 3
    df['Four of a kind or Full house'] = df['Four of a kind or Full house'].fillna(False)
    df['Two pairs or Three of a kind'] = df['Two pairs or Three of a kind'].fillna(False)
    dff = np.array(df[['C1', 'C2', 'C3', 'C4', 'C5']])
    a = []
    for i in range(dff.shape[0]):
        u, v  = np.unique(dff[i], return_counts=True)
        a.append(4 in v)
    df['Four of a kind'] = np.array(a)
    df['Full house'] = (df['Four of a kind']==False) & (df['Four of a kind or Full house'] ==True)
    df = df.drop('Four of a kind or Full house', axis =1)
    a = []
    for i in range(dff.shape[0]):
        u, v  = np.unique(dff[i], return_counts=True)
        a.append(set(v) == set([1,1,3]))
    df['Three of a kind'] = np.array(a)
    df['Two pairs'] = (df['Three of a kind']==False) & (df['Two pairs or Three of a kind'] ==True)
    df = df.drop('Two pairs or Three of a kind', axis =1)
    df = df.drop(['S1', 'S2','S3','S4','S5','C1','C2','C3','C4','C5'], axis = 1)
    return df

In [ ]:
df.head(15)



In [ ]:
df = preproc(df)

In [ ]:
df.head(15)

In [ ]:
x = df.drop('hand', axis =1)
y = df['hand']
x = np.array(x).astype(int)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(x, y)
x_test = test_data
x_test = x_test.drop('id', axis = 1)
x_test = preproc(x_test)


In [ ]:
x_test = np.array(x_test).astype(int)
predictions = model.predict(x_test).astype(int)
output = pd.DataFrame({'id': test_data.id, 'hand': predictions})
output.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")
output